# Best Models With Food Data

## Common Part
This part has the functions that are needed to perform the models

In [ ]:
import sys
sys.path.append('Src/')
from data_modificado import * 
from train_2 import * 
from transfer_learning import * 
from transfer_learning_FI import * 
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns

def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else:
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] 
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
        else:
            domain_shape=None
        models = autoencoder(bioma_shape=bioma_shape,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             latent_space=latent_space,
                             bioma_layers=layers, 
                             domain_layers=domain_layers,
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)

        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

#-----------------------------
def save_predicted_otu_table_and_latent(pred,pred_latent,sample_names,otu_names,suffix=''):
    df_otu = pd.DataFrame(pred, index=sample_names, columns=otu_names)
    df_otu.T.to_csv('Results/otus_'+suffix+'.tsv', index=True, header=True, sep='\t')

    df_latent = pd.DataFrame(pred_latent, index=sample_names)
    df_latent.T.to_csv('Results/latent_'+suffix+'.tsv', index=True, sep='\t')
    
    return df_otu, df_latent

#Compute metrics by OTU
# Absolute abundance transformed to TSS (with epsilon=1E-6)
def transform_to_rel_abundance(dataset):
    epsilon=1E-6
    sum_per_sample = dataset.sum(axis=1)
    num_samples = sum_per_sample.shape
    num_OTUs = np.shape(dataset)[-1] 
    sum_per_sample = sum_per_sample + (num_OTUs * epsilon)
    dividend=dataset+epsilon
    dataset_rel_abund = np.divide(dividend,sum_per_sample[:,None])
    return dataset_rel_abund
def compute_relative_squared_error(actual,pred_domain):
    rse_otu=np.zeros(actual.shape[1],dtype=np.float32)
    actual=actual.transpose()
    pred=pred_domain.transpose()
    for i, (act_otu,pred_otu) in enumerate(zip(actual,pred)):
        mean_otu = act_otu.mean()
        div_up=((pred_otu-act_otu)**2).sum()
        div_down=((pred_otu-mean_otu)**2).sum()
        rse_otu[i]=div_up/div_down
    return rse_otu, np.sqrt(rse_otu)

def save_errors_per_OTU(RSE,RRSE,otu_names,suffix=''):
    df_RSE = pd.DataFrame(RSE, index=otu_names, columns=['RSE'])
    df_RRSE = pd.DataFrame(RRSE, index=otu_names, columns=['RRSE'])
    df_error = df_RSE.join(df_RRSE)
    df_error.to_csv(suffix+'.tsv', index=True, header=True, sep='\t')
    
    return df_error

#-------------------------------
def FI():
    metric_results, _ = test_model_tl_noEnsemble_FI(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)
    error_original = metric_results["BrayCurtis"][0]
    metricas = dict()
    for i, variable in enumerate(nombres_metadatos):
        df1 = df_domain_test
        valores_BrayCurtis = []
        for j in range(0,10,1):
            df1[variable] = np.random.permutation(df_domain_test[variable].values)
            data_domain_test_nueva = df1.to_numpy()
            metric_results, _ = test_model_tl_noEnsemble_FI(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test_nueva)
            #valores_BrayCurtis.append(metric_results["BrayCurtis"][0]/error_original)
            valores_BrayCurtis.append(metric_results["BrayCurtis"][0]-error_original)
        metricas[variable] = valores_BrayCurtis
    return metricas

In [ ]:
def save_data(encoder, model_latent, name_type_model, taxonomical_rank, data_microbioma_test, df_microbioma_test):
    encoder_biome = encoder
    encoder_domain = model_latent

    #run prediction test set from microbiome, i.e, reconstructed
    # Input only domain (i.e. environmental features)
    pred_latent_biome = encoder_biome.predict(data_microbioma_test)
    pred_biome = decoder.predict(pred_latent_biome)

    _, _ = save_predicted_otu_table_and_latent(pred_biome,pred_latent_biome,df_microbioma_test.index,df_microbioma_test.columns,'reconstAEfromBiome_'+name_type_model+"_"+taxonomical_rank)

    #run prediction test set from domain, i.e., diet features
    # Input only domain (i.e. environmental features)
    pred_latent = encoder_domain.predict(data_domain_test)
    pred_domain = decoder.predict(pred_latent)
    df_pred_otu, df_pred_latent = save_predicted_otu_table_and_latent(pred_domain,pred_latent,df_microbioma_test.index,df_microbioma_test.columns,'predFromDomain_'+name_type_model+"_"+taxonomical_rank)
    
    #--------------------
    actual_array = transform_to_rel_abundance(data_microbioma_test)
    RSE_perOTU, RRSE_perOTU  = compute_relative_squared_error(actual_array, pred_domain)

    df_error_perOTU = save_errors_per_OTU(RSE_perOTU,RRSE_perOTU,df_microbioma_test.columns,'Results/errors_perOTU_'+name_type_model+"_"+taxonomical_rank)
    
    #-----------------
    metrics = FI()
    tabla_FI = pd.DataFrame.from_dict(metrics)
    tabla_FI.to_csv("Results/FI_"+name_type_model+'_'+taxonomical_rank+'.csv',index=False)

--------------------------
--------------------------
# 37 variables Latent OTU

In [ ]:
def model_fn_latent():
    in_layer = layers.Input(shape=(data_domain_train.shape[1],))
    net = layers.Dense(128, activation='sigmoid')(in_layer)
    net = layers.Dense(64, activation='sigmoid')(net)
    net = layers.Dense(32, activation='sigmoid')(net)
    net = layers.Dense(16, activation='sigmoid')(net)
    out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net)
    model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
    model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

### Specie

In [ ]:
#Load Data
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Specie.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Specie", data_microbioma_test, df_microbioma_test)

------------------
### Genus

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Genus.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Genus", data_microbioma_test, df_microbioma_test)

-----------------
### Family

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Family.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Family", data_microbioma_test, df_microbioma_test)

-----------------
### Order

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Order.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Order", data_microbioma_test, df_microbioma_test)

-----------------
### Class

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Class.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Class", data_microbioma_test, df_microbioma_test)

----------------
### Phylum

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Phylum.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37OTU", "Phylum", data_microbioma_test, df_microbioma_test)

--------------------------
--------------------------
# 37 variables Latent Combined

In [ ]:
def model_fn_latent():
    in_layer = layers.Input(shape=(data_domain_train.shape[1],))
    net = layers.Dense(128, activation='sigmoid')(in_layer)
    net = layers.Dense(64, activation='sigmoid')(net)
    net = layers.Dense(32, activation='sigmoid')(net)
    net = layers.Dense(16, activation='sigmoid')(net)
    out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net)
    model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
    model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

### Specie

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Specie.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Specie", data_microbioma_test, df_microbioma_test)

-------------------
### Genus

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Genus.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Genus", data_microbioma_test, df_microbioma_test)

----------
### Family

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Family.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Family", data_microbioma_test, df_microbioma_test)

----------------
### Order

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Order.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Order", data_microbioma_test, df_microbioma_test)

------------------
### Class

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Class.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Class", data_microbioma_test, df_microbioma_test)

--------------
### Phylum

In [ ]:
nombres_metadatos = ["F_TOTAL","F_CITMLB","F_OTHER","F_JUICE","V_TOTAL","V_DRKGR","V_REDOR_TOTAL","V_REDOR_TOMATO","V_REDOR_OTHER","V_STARCHY_TOTAL","V_STARCHY_POTATO","V_STARCHY_OTHER","V_OTHER",\
                     "V_LEGUMES","G_TOTAL","G_WHOLE","G_REFINED","PF_TOTAL","PF_MPS_TOTAL","PF_MEAT","PF_CUREDMEAT","PF_ORGAN","PF_POULT","PF_SEAFD_HI","PF_SEAFD_LOW","PF_EGGS","PF_SOY","PF_NUTSDS",\
                     "PF_LEGUMES","D_TOTAL","D_MILK","D_YOGURT","D_CHEESE","OILS","SOLID_FATS","ADD_SUGARS","A_DRINKS"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Phylum.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "37Combined", "Phylum", data_microbioma_test, df_microbioma_test)

----------------------
----------------------
# 22 variables Latent OTU

In [ ]:
def model_fn_latent():
    in_layer = layers.Input(shape=(data_domain_train.shape[1],))
    net = layers.Dense(128, activation='sigmoid')(in_layer)
    net = layers.Dense(64, activation='sigmoid')(net)
    net = layers.Dense(32, activation='sigmoid')(net)
    net = layers.Dense(16, activation='sigmoid')(net)
    out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net)
    model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
    model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

### Specie

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Specie.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Specie", data_microbioma_test, df_microbioma_test)

---------------
### Genus

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Genus.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Genus", data_microbioma_test, df_microbioma_test)

---------------
### Family

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Family.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Family", data_microbioma_test, df_microbioma_test)

---------------
### Order

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Order.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Order", data_microbioma_test, df_microbioma_test)

---------------
### Class

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Class.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Class", data_microbioma_test, df_microbioma_test)

---------------
### Phylum

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Phylum.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=96,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=None,
                                                               show_results=False,
                                                               device='/CPU:0')

#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22OTU", "Phylum", data_microbioma_test, df_microbioma_test)

-----------------
-----------------
# 22 variables Latent Combined

In [ ]:
def model_fn_latent():
    in_layer = layers.Input(shape=(data_domain_train.shape[1],))
    net = layers.Dense(128, activation='relu')(in_layer)
    net = layers.Dense(64, activation='relu')(net)
    net = layers.Dense(32, activation='relu')(net)
    net = layers.Dense(16, activation='relu')(net)
    out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net)
    model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
    model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

### Specie

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Specie.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Specie", data_microbioma_test, df_microbioma_test)

---------------
### Genus

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Genus.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Genus", data_microbioma_test, df_microbioma_test)

--------------
### Family

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Family.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Family", data_microbioma_test, df_microbioma_test)

-------------
### Order

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Order.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Order", data_microbioma_test, df_microbioma_test)

-----------------
### Class

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Class.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Class", data_microbioma_test, df_microbioma_test)

----------------
### Phylum

In [ ]:
nombres_metadatos = ['F_TOTAL', 'V_TOTAL', 'V_REDOR_TOTAL', 'V_STARCHY_TOTAL', 'V_LEGUMES', 'G_TOTAL', 'PF_TOTAL', 'PF_MEAT', 'PF_CUREDMEAT', 'PF_ORGAN', 'PF_POULT','PF_SEAFD_HI', 'PF_SEAFD_LOW', 'PF_EGGS', 'PF_SOY', 'PF_NUTSDS', 'D_TOTAL', 'D_YOGURT', 'OILS', 'SOLID_FATS', 'ADD_SUGARS', 'A_DRINKS']
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='otu_table_Phylum.csv',metadata_filename='FoodMetadata.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0,
                                                               epochs=100,
                                                               batch_size=64,
                                                               learning_rate=0.001,
                                                               optimizer=optimizers.Adam,
                                                               learning_rate_scheduler=None,
                                                               input_transform=Percentage,
                                                               output_transform=tf.keras.layers.Softmax,
                                                               reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None),
                                                               latent_space=15,
                                                               layers=[512,256],
                                                               activation='tanh',
                                                               activation_latent='tanh',
                                                               data_microbioma_train=data_microbioma_train,
                                                               data_domain_train=data_domain_train,
                                                               show_results=False,
                                                               device='/CPU:0')


#Save encoder and decoder
model, encoder, _ ,decoder = models[0]
    
#predict latent space
latent_train = encoder.predict(data_microbioma_train)

#train the encoder
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                  latent_train,
                                                  latent_train,
                                                  data_domain_train,
                                                  data_domain_train,
                                                  epochs=100,
                                                  batch_size=16,
                                                  verbose=-1)

#evaluate the model
latent_test = encoder.predict(data_microbioma_test)
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#Save predictions, error per OTU and model FI
save_data(encoder, model_latent, "Food22Combined", "Phylum", data_microbioma_test, df_microbioma_test)